In [788]:
%load_ext autoreload
%autoreload 2

import numpy as np
from investment_planner import calculateTransitionPropabilitiesForAllPorfolios, generateGrid, calculateTransitionPropabilities, calculateTransitionPropabilitiesForGoals, get_goals_strategies
from scipy.stats import norm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [789]:

portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])
W0 = np.array([90,95,100,105])
W1 = np.array([100,103,105,110])
infusions = 2
cost = 3
h = 1

values = [0,0,100,100]

#propabilities = calculateTransitionPropabilitiesForAllPorfolios(portfolios, W0, W1, infusions, cost, h)
propabilities = np.array([[[0.451, 0.334, 0.197, 0.018],
                                    [0.271, 0.27,  0.258, 0.2  ],
                                    [0.255, 0.256, 0.253, 0.237]],

                                    [[0.14,  0.307, 0.366, 0.187],
                                    [0.221, 0.254, 0.266, 0.259],
                                    [0.24,  0.251, 0.255, 0.254]],

                                    [[0.017, 0.105, 0.244, 0.634],
                                    [0.177, 0.234, 0.268, 0.322],
                                    [0.227, 0.246, 0.256, 0.271]],

                                    [[0.001, 0.017, 0.075, 0.908],
                                    [0.14,  0.211, 0.262, 0.387],
                                    [0.215, 0.241, 0.257, 0.288]]])
print(np.round(propabilities, 3))
values = propabilities*values
np.round(np.amax(values.sum(2),1),3)
#values.sum(2)


[[[0.451 0.334 0.197 0.018]
  [0.271 0.27  0.258 0.2  ]
  [0.255 0.256 0.253 0.237]]

 [[0.14  0.307 0.366 0.187]
  [0.221 0.254 0.266 0.259]
  [0.24  0.251 0.255 0.254]]

 [[0.017 0.105 0.244 0.634]
  [0.177 0.234 0.268 0.322]
  [0.227 0.246 0.256 0.271]]

 [[0.001 0.017 0.075 0.908]
  [0.14  0.211 0.262 0.387]
  [0.215 0.241 0.257 0.288]]]


array([49. , 55.3, 87.8, 98.3])

In [790]:
portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057], [0.0886, 0.1954]])
W0 = 100
T = 12
#infusions = [0,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000]
infusions = np.zeros(13)
goals = [0,0,0,0,0,0,0,100,0,0,0,0,150]
iMax = 281

grid = generateGrid(W0,T,iMax, infusions, goals, portfolios[0,0], portfolios[0,1], portfolios[2,0], portfolios[2,1])
grid = grid.T

TypeError: list indices must be integers or slices, not tuple

In [ ]:
W0 = 100
meanMax = 0.0886
stdMin = 0.0374
stdMax = 0.1954
t =11

W0*np.exp((meanMax-(stdMin**2/2))*t + 3*stdMax*np.sqrt(t))

1837.723

In [ ]:
k = np.array([[105,100],[110,200]])
wealthInT = np.array([90,95,100,105,110])
Wk = np.tile(wealthInT, (2,1))
ki = np.tile(k, (2, 5,1))
#np.where(Wk >= k[:,0], k[:,1], 0)
ki

array([[[105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200]],

       [[105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200]]])

In [ ]:
WT1 = np.array([90,95,100,105,110])
WT1P = WT1 * [0.2,0.35, 0.2 ,0.15,0.1]
WT1P.reshape((5,1))

array([[18.  ],
       [33.25],
       [20.  ],
       [15.75],
       [11.  ]])

In [ ]:
arr = np.array([[1,2,3,4],[1,2,3]])
arr2 = np.array([1,2])
result = arr*arr2
result

C:\Users\matsz\AppData\Local\Temp\ipykernel_28528\4293703242.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.array([[1,2,3,4],[1,2,3]])


array([list([1, 2, 3, 4]), list([1, 2, 3, 1, 2, 3])], dtype=object)

In [ ]:
def get_ef_port_analytically(rets, covm, targ):
    """Solve for the efficient frontier weights for a given expected return
    vector `rets`, covariance matrix `covm`, and expected portfolio return
    `targ`.
    """
    N = rets.shape[0]
    u = np.array([targ, 1])[:, None]
    U = np.vstack([rets, np.ones_like(rets)]).T

    covm_inv = np.linalg.solve(covm, np.eye(N))
    M        = U.T @ covm_inv @ U
    M_inv    = np.linalg.solve(M, np.eye(2))
    weights  = covm_inv @ U @ M_inv @ u

    return weights


In [ ]:
cov_tab = np.array([[0.0002, -0.00],
                    [-0.00, 0.0013]])
means = np.array([0.0307, 0.1094])

mu = 0.0744
std = 0.0712

w = get_ef_port_analytically(means,cov_tab,mu)

w

array([[0.44472681],
       [0.55527319]])

In [ ]:
arr = np.empty((3,3))

arr[:] = np.nan
arr




array([[nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan]])

In [ ]:
mean = 0.0886
std = 0.1954
W0 = 105
W1 = np.array([105,110,115])
h= 1
np.round(norm.pdf((np.log(W1/(W0))-(mean-0.5*std**2)*h)/(std*np.sqrt(h))),3)

#100 [0.39670583, 0.39547966, 0.37397359]
#105 [0.37448256, 0.39619069, 0.39654302]


array([0.374, 0.396, 0.397])

In [798]:
portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])
W0 = np.array([90,95,100,105])
W1 = np.array([100,103,105,110])
infusions = 2
cost = 50
h = 1

values = [0,0,100,100]

propabilities = calculateTransitionPropabilitiesForAllPorfolios(portfolios, W0, W1, infusions, h)
''' propabilities = np.array([[[0.451, 0.334, 0.197, 0.018],
                                    [0.271, 0.27,  0.258, 0.2  ],
                                    [0.255, 0.256, 0.253, 0.237]],

                                    [[0.14,  0.307, 0.366, 0.187],
                                    [0.221, 0.254, 0.266, 0.259],
                                    [0.24,  0.251, 0.255, 0.254]],

                                    [[0.017, 0.105, 0.244, 0.634],
                                    [0.177, 0.234, 0.268, 0.322],
                                    [0.227, 0.246, 0.256, 0.271]],

                                    [[0.001, 0.017, 0.075, 0.908],
                                    [0.14,  0.211, 0.262, 0.387],
                                    [0.215, 0.241, 0.257, 0.288]]]) '''
print(np.round(propabilities, 3))
values = propabilities*values
np.round(np.amax(values.sum(2),1),3)

[[[0.657 0.247 0.093 0.003]
  [0.31  0.357 0.28  0.052]
  [0.068 0.227 0.355 0.351]
  [0.006 0.053 0.161 0.78 ]]

 [[0.304 0.277 0.25  0.169]
  [0.25  0.264 0.262 0.223]
  [0.203 0.246 0.268 0.283]
  [0.162 0.225 0.266 0.348]]

 [[0.264 0.258 0.251 0.226]
  [0.249 0.254 0.254 0.244]
  [0.235 0.249 0.255 0.261]
  [0.222 0.244 0.256 0.278]]]


array([94.097, 61.372, 53.396])

In [803]:
def get_portfolios_strategies(VT1, probabilities):
    Vt = VT1 * probabilities
    sums = Vt.sum(2)
    maxes = np.amax(sums,0)
    portfolios_ids = np.argmax(sums,0)    
    chosen_propabilities = np.take_along_axis(probabilities.transpose(1,0,2),portfolios_ids.reshape(6,1,1),1).squeeze(1)
    return portfolios_ids, maxes, chosen_propabilities

portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])
WT = np.array([49,50,60,80,100,105])
WT1 = np.array([90,95,100,103,105,110])
VT1 = [0,0,0,100,100,100]
c = 50
#goals = [[50, 80], [30, 50],]
VTc0 = np.array([29.014, 29.55, 77.286, 95.997])

probabilities = calculateTransitionPropabilitiesForAllPorfolios(portfolios,WT,WT1,5,1)
print(np.round(probabilities,3))
""""
portfolios_strategies, VT, chosen_propabilities, V = get_portfolios_strategies(VT1,probabilities)
print(portfolios_strategies)
print(VT)
print(np.round(chosen_propabilities,3))
print(np.round(V,3)) 
"""
porfolios_strategies, VT, chosen_propabilities = get_portfolios_strategies(VT1,probabilities)
print(np.round(porfolios_strategies,1))
print(np.round(VT,1))
print(np.round(chosen_propabilities,2))


[[[1.    0.    0.    0.    0.    0.   ]
  [1.    0.    0.    0.    0.    0.   ]
  [1.    0.    0.    0.    0.    0.   ]
  [0.769 0.221 0.01  0.001 0.    0.   ]
  [0.    0.    0.017 0.105 0.244 0.633]
  [0.    0.    0.001 0.017 0.075 0.908]]

 [[0.91  0.082 0.006 0.001 0.    0.   ]
  [0.901 0.089 0.008 0.002 0.001 0.   ]
  [0.769 0.178 0.035 0.012 0.006 0.001]
  [0.281 0.255 0.18  0.132 0.103 0.049]
  [0.03  0.08  0.158 0.208 0.238 0.286]
  [0.016 0.053 0.131 0.196 0.244 0.36 ]]

 [[0.472 0.243 0.121 0.078 0.058 0.027]
  [0.461 0.244 0.124 0.081 0.061 0.029]
  [0.353 0.237 0.151 0.113 0.092 0.054]
  [0.196 0.192 0.176 0.161 0.151 0.124]
  [0.107 0.141 0.171 0.185 0.193 0.204]
  [0.092 0.129 0.167 0.188 0.2   0.224]]]
[2 2 2 2 0 0]
[16.4 17.2 25.9 43.6 98.3 99.9]
[[0.47 0.24 0.12 0.08 0.06 0.03]
 [0.46 0.24 0.12 0.08 0.06 0.03]
 [0.35 0.24 0.15 0.11 0.09 0.05]
 [0.2  0.19 0.18 0.16 0.15 0.12]
 [0.   0.   0.02 0.1  0.24 0.63]
 [0.   0.   0.   0.02 0.07 0.91]]


In [808]:
def __calculateWtc(WT, goals_costs, infusion):        
    k = len(goals_costs)
    cf = goals_costs - infusion
    Wtc = np.tile(WT,(k,1)) - cf.reshape((k,1))
    return Wtc
goals = np.array([[40,10],[49, 15],[60, 30]])
WT = np.array([49,50,60,80,100,105])
infusion = 5
__calculateWtc(WT,goals[:,0],infusion)

array([[14, 15, 25, 45, 65, 70],
       [ 5,  6, 16, 36, 56, 61],
       [-6, -5,  5, 25, 45, 50]])

In [884]:
def __calculate_drift(goal_porfolio_strategies, portfolios, h=1):
    portfolios_measures = np.take(portfolios, goal_porfolio_strategies, axis=0)
    b = (portfolios_measures[:,:,0]-0.5*portfolios_measures[:,:,1]**2)*h
    c = portfolios_measures[:,:,1]*np.sqrt(h)
    return portfolios_measures, b, c

goal_porfolio_strategies = np.array([[2,2,1,0],
                                    [2,2,2,1]])

portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])

__calculate_drift(goal_porfolio_strategies, portfolios)

#b result porfolio 2, 0.06950942




(array([[[0.0886    , 0.1954    ],
         [0.0886    , 0.1954    ],
         [0.07059443, 0.103057  ],
         [0.0526    , 0.0374    ]],
 
        [[0.0886    , 0.1954    ],
         [0.0886    , 0.1954    ],
         [0.0886    , 0.1954    ],
         [0.07059443, 0.103057  ]]]),
 array([[0.06950942, 0.06950942, 0.06528406, 0.05190062],
        [0.06950942, 0.06950942, 0.06950942, 0.06528406]]),
 array([[0.1954  , 0.1954  , 0.103057, 0.0374  ],
        [0.1954  , 0.1954  , 0.1954  , 0.103057]]))

In [894]:
from investment_planner import __get_porfolios_strategy_for_wealth_values

def calculateTransitionPropabilitiesForGoals(WTc, Wt1, h,portfolios_wtc):
    i0 = WTc.shape[1]
    i1 = len(Wt1)    
    k = WTc.shape[0]

    portfolios_measures = portfolios_wtc.reshape(k*i0,2)  
    b = (portfolios_measures[:,0]-0.5*portfolios_measures[:,1]**2)*h
    b = b.reshape(k*i0,1)
    c = portfolios_measures[:,1]*np.sqrt(h)
    c = c.reshape(k*i0,1)
    result = np.zeros((k*i0,i1))

    Wtc = WTc.reshape((k*i0,1))
    Wt1k = np.tile(Wt1, (k*i0,1))
    
    np.divide(Wt1k, Wtc, out=result, where=Wtc>0)
    np.log(result,out=result, where=result>0)
    result = np.where(result > 0, result - b, 0)
    result = result / c
    result = np.where(result > 0, norm.pdf(result), 0)

    #result = np.divide(result,np.expand_dims(result.sum(1), axis=1),where=result>0)
    result = result.reshape(k,i0,i1) 
    return result

goals = np.array([[5,5],[49, 15],[60, 30]])
WT = np.array([49,50,60,80,100,105])
infusion = 5
porfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])
Wtc = np.array([[44, 45, 55, 75, 95, 100],
                [ 5,  6, 16, 36, 56, 61],
                [-6, -5,  5, 25, 45, 50]])

portfolios_for_goals = np.array([[[0.0886, 0.1954],
                                  [0.0886, 0.1954],
                                  [0.0886, 0.1954],
                                  [0.0886, 0.1954],
                                  [0.0526, 0.0374],
                                  [0.0526, 0.0374]],                        
                                 [[0.0886, 0.1954],
                                  [0.0886, 0.1954],
                                  [0.0886, 0.1954],
                                  [0.0886, 0.1954],
                                  [0.0886, 0.1954],
                                  [0.0886, 0.1954]],
                                  [[0.0886, 0.1954],
                                  [0.0886, 0.1954],
                                  [0.0886, 0.1954],
                                  [0.0886, 0.1954],
                                  [0.0886, 0.1954],
                                  [0.0886, 0.1954]],
                                  ])

prob = calculateTransitionPropabilitiesForGoals(Wtc, WT1, 1,portfolios_for_goals)
np.round(prob, 3)


array([[[0.002, 0.001, 0.   , 0.   , 0.   , 0.   ],
        [0.002, 0.001, 0.   , 0.   , 0.   , 0.   ],
        [0.038, 0.02 , 0.01 , 0.007, 0.005, 0.002],
        [0.338, 0.277, 0.214, 0.179, 0.157, 0.11 ],
        [0.   , 0.   , 0.   , 0.296, 0.174, 0.016],
        [0.   , 0.   , 0.   , 0.   , 0.   , 0.203]],

       [[0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
        [0.047, 0.025, 0.013, 0.009, 0.007, 0.003],
        [0.105, 0.064, 0.038, 0.027, 0.021, 0.012]],

       [[0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
        [0.002, 0.001, 0.   , 0.   , 0.   , 0.   ],
        [0.012, 0.005, 0.002, 0.001, 0.001, 0.   ]]])

In [896]:
def __prob(W0, W1, mean, std, Infusion, Cost, h):
    return norm.pdf(((np.log(W1/(W0+Infusion+Cost))-(mean-0.5*std**2)*h))/(std*np.sqrt(h)))

__prob(95,100,0.0526,0.0374,0,0,1)

0.399

In [ ]:
def __get_porfolio_strategy_for_wealth_values(WT, wealth_value, porfolios_strategies):
    k = wealth_value.shape[0]
    i = wealth_value.shape[1]
    wc = wealth_value.reshape(k*i,1)
    wg = np.tile(WT,(k*i,1))
    difference = np.absolute(wg - wc)
    index = np.argmin(difference, axis=1)
    index = index.reshape(k,i)
    return np.take(porfolios_strategies,index)

porfolios_strategies = [2,2,2,1,0,0]
WT = np.array([49,50,60,80,100,105])
goals = np.array([[5,5],[20,10],[60, 30]])
Wtc = np.tile(WT,(len(goals),1)) - goals[:,0].reshape(len(goals),1)
#result [[2,2,2,1,0,0],
#        [2,2,2,2,1,1],
#        [2,2,2,2,2,2]
print(WT)
print(Wtc)

__get_porfolio_strategy_for_wealth_values(WT, Wtc, porfolios_strategies)


[ 49  50  60  80 100 105]
[[ 44  45  55  75  95 100]
 [ 29  30  40  60  80  85]
 [-11 -10   0  20  40  45]]


array([[2, 2, 2, 1, 0, 0],
       [2, 2, 2, 2, 1, 1],
       [2, 2, 2, 2, 2, 2]])

In [832]:
from investment_planner import __calculateWtc, __get_porfolios_strategy_for_wealth_values

%precision 3
def get_goals_strategies(goals, infusion, Wt, Wt1, VTK1, portfolios, h=1):
    k = len(goals)
    i = len(Wt)
    
    probabilities = calculateTransitionPropabilitiesForAllPorfolios(portfolios,Wt,Wt1,infusion,1)
    portfolios_strategies, VTk0, chosen_propabilities = get_portfolios_strategies(VTK1,probabilities)
    
    probabilities_kc = np.zeros((k+1, i, len(Wt1)))
    values = np.zeros((k+1, i)) 
      
    values[0] = VTk0
    probabilities_kc[0] = chosen_propabilities
    
    ''' for k in range(k):
        for i in range(i):
            value_index = __get_value_index(Wt, Wtc[k,i])
            portfolio_strategy = portfolios_strategies[value_index]
            probabilities = calculateTransitionPropabilities(portfolios[portfolio_strategy],Wtc[k,i],Wt1,infusion,0)
            values[k+1,i] = (probabilities * VTK1).sum()+ goals[k,1]
            porfolios_strategies[k,i] = portfolio_strategy
            propabilities_kc[k+1,i] = probabilities '''
    
    Wtc = __calculateWtc(Wt,goals[:,0],infusion)
    psWtc = __get_porfolios_strategy_for_wealth_values(Wt,Wtc,porfolios_strategies)
    probabilities_kc[1:] = calculateTransitionPropabilitiesForGoals(Wtc,Wt1,1,psWtc)
    values[1:] = (probabilities_kc[1:] * VTK1).sum(2)
                  
                        
    strategies = values.argmax(0)
    chosen_goal_propabilities = np.take_along_axis(probabilities_kc,np.expand_dims(strategies,axis=(0,1)),1)
    return strategies, portfolios_strategies, values,  probabilities_kc #np.squeeze(chosen_goal_propabilities)

goals = np.array([[40,10],[49, 15],[60, 30]])
#goals = np.array([[0,0]])
infusion = 5

Wt = WT

goal_strategies, portfolio_strategies, values, probabilities = get_goals_strategies(goals, infusion, WT,WT1, VT1, portfolios)


print(goal_strategies)
print(portfolio_strategies)
print(np.round(values,3))
print(np.round(probabilities,3))



[1 1 1 1 0 0]
[2 2 2 2 0 0]
[[16.384 17.2   25.916 43.611 98.276 99.897]
 [44.57  44.761 46.176 47.803 48.819 49.024]
 [41.724 42.227 44.94  47.186 48.408 48.644]
 [ 0.     0.    41.724 46.176 47.803 48.095]]
[[[0.472 0.243 0.121 0.078 0.058 0.027]
  [0.461 0.244 0.124 0.081 0.061 0.029]
  [0.353 0.237 0.151 0.113 0.092 0.054]
  [0.196 0.192 0.176 0.161 0.151 0.124]
  [0.    0.    0.017 0.105 0.244 0.633]
  [0.    0.    0.001 0.017 0.075 0.908]]

 [[0.204 0.184 0.167 0.157 0.151 0.137]
  [0.202 0.183 0.167 0.157 0.152 0.138]
  [0.192 0.179 0.167 0.16  0.156 0.146]
  [0.181 0.174 0.167 0.163 0.161 0.154]
  [0.174 0.171 0.167 0.165 0.163 0.16 ]
  [0.173 0.17  0.167 0.165 0.164 0.161]]

 [[0.225 0.192 0.165 0.151 0.143 0.124]
  [0.221 0.191 0.165 0.152 0.144 0.126]
  [0.201 0.183 0.167 0.158 0.152 0.139]
  [0.185 0.176 0.167 0.162 0.159 0.151]
  [0.177 0.172 0.167 0.164 0.162 0.157]
  [0.175 0.171 0.167 0.165 0.163 0.159]]

 [[0.    0.    0.    0.    0.    0.   ]
  [0.    0.    0.    0.  

In [ ]:
i = 6
k = 3
#Wtc = - np.tile(Wt,(k,1)) - np.repeat(goals[:,0],i).reshape((k,i))
#Wtc[Wtc < 0] = 0
#np.repeat(goals[:,0],i).reshape((k,i))
np.tile(Wt,(k,1)) - np.repeat(goals[:,0],i).reshape((k,i))


array([[  9,  10,  20,  40,  60,  65],
       [  0,   1,  11,  31,  51,  56],
       [-11, -10,   0,  20,  40,  45]])

In [ ]:
WT1 = np.array([90,95,100,103,105,110])
np.round(calculateTransitionPropabilities(portfolios[2],40,WT1,3,0),3)

array([0.606, 0.226, 0.083, 0.045, 0.03 , 0.011])

```
expected values array([24.794, 29.794, 29.794, 44.794, 95.456, 99.669])
array([[14.794, 15.581, 24.116, 41.942, 95.456, 99.669],
       [24.794, 24.794, 24.794, 24.794, 34.116, 34.116],
       [14.794, 29.794, 29.794, 29.794, 30.581, 39.116],
       [14.794, 15.581, 24.116, 44.794, 44.794, 44.794]])

```

## TODO
- sprawdzić wrażliwość na rożne inputy
- W0 jest większe niż cele

In [ ]:
from investment_planner import InvestmentPlanner 

investment_planner = InvestmentPlanner()

goals = np.array([[[0,0]],[[0,0]],[[0,0]],[[0,0]],[[0,0]],[[90,60]],[[0,0]],[[0,0]],[[0,0]],[[0,0]],[[120,100]]])
portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])
investment_planner.set_params(10, 100, 10, 1, goals, portfolios)

glide_path = investment_planner.glide_paths
wealth_grid = investment_planner.grid
probabilitiesT = investment_planner.probabilitiesT
goal_strategies = investment_planner._goal_strategies

#TODO refactor investment planner - chosen propabilieis to get portfolio_propabilities


ValueError: cannot reshape array of size 500 into shape (6,1,1)

In [ ]:
WTt = np.repeat(10,1200)
goals = np.array([[40,10],[49, 15],[60, 30]])

In [ ]:
def calculateTransitionPropabilitiesForGoals(Wt, Wt1, infusion, h, goal_costs, portfolioMeasures):
    i0 = len(Wt)
    i1 = len(Wt1)    
    k = len(goal_costs)
    cf = goal_costs - infusion
    mean = portfolioMeasures[0]
    std = portfolioMeasures[1]
    b = ((mean-0.5*std**2)*h)/(std*np.sqrt(h))
    result = np.zeros((k*i0,i1))
    
    Wtc = np.tile(Wt,(k,1)) - cf.reshape((k,1))
    Wtc = Wtc.reshape((k*i0,1))
    Wt1k = np.tile(Wt1, (k*i0,1))
    
    np.divide(Wt1k, Wtc, out=result, where=Wtc>0)
    np.log(result,out=result, where=result>0)
    result = np.where(result > 0, result+ b, 0)
    result = np.where(result > 0, norm.pdf(result), 0)

    result = np.divide(result,np.expand_dims(result.sum(1), axis=1),where=result>0)
    result = result.reshape(k,i0,i1)

    return result
   

portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])
WT = np.array([49,50,60,80,100,105])
#WT = np.array([100])
WT1 = [90,95,100,103,105,110]

goals = np.array([[40,10],[49, 15],[60, 30]])
#goals = np.array([[49, 15],[60, 30]])
#goals = np.array([[60,0]])
infusion = 5
h = 1

probabilities = calculateTransitionPropabilitiesForGoals(WT, WT1, infusion, h, goals[:,0], [0.1,0.2])

print(np.round(probabilities,3))
probabilities.sum(1)



[[[0.212 0.187 0.166 0.155 0.148 0.132]
  [0.211 0.187 0.166 0.155 0.148 0.133]
  [0.2   0.182 0.167 0.158 0.153 0.14 ]
  [0.189 0.177 0.167 0.161 0.157 0.148]
  [0.181 0.174 0.167 0.163 0.16  0.154]
  [0.18  0.174 0.167 0.163 0.161 0.155]]

 [[0.234 0.195 0.164 0.149 0.139 0.118]
  [0.23  0.194 0.165 0.15  0.141 0.121]
  [0.209 0.186 0.166 0.156 0.149 0.134]
  [0.193 0.179 0.167 0.16  0.156 0.145]
  [0.184 0.176 0.167 0.162 0.159 0.152]
  [0.183 0.175 0.167 0.163 0.16  0.153]]

 [[0.    0.    0.    0.    0.    0.   ]
  [0.    0.    0.    0.    0.    0.   ]
  [0.234 0.195 0.164 0.149 0.139 0.118]
  [0.2   0.182 0.167 0.158 0.153 0.14 ]
  [0.189 0.177 0.167 0.161 0.157 0.148]
  [0.186 0.177 0.167 0.162 0.158 0.15 ]]]


array([[1.17265765, 1.08174504, 1.0004663 , 0.95577458, 0.92752447,
        0.86183196],
       [1.23281172, 1.10540205, 0.99677676, 0.9391091 , 0.90339422,
        0.82250615],
       [0.80898429, 0.73189712, 0.66534571, 0.6296638 , 0.60743232,
        0.55667676]])

```
[[0.    0.    0.    0.    0.    0.   ]
 [0.001 0.001 0.001 0.001 0.    0.   ]
 [0.016 0.014 0.012 0.012 0.011 0.01 ]
 [0.091 0.083 0.076 0.072 0.069 0.064]
 [0.177 0.165 0.154 0.148 0.144 0.134]
 [0.197 0.184 0.173 0.166 0.162 0.152]
 [0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.   ]
 [0.046 0.041 0.037 0.034 0.033 0.03 ]
 [0.131 0.121 0.111 0.106 0.103 0.095]
 [0.152 0.141 0.131 0.125 0.122 0.113]]
 ```

In [ ]:
arr =  np.array([0.197, 0.184, 0.173, 0.166, 0.162, 0.152])
res = arr/ arr.sum()
np.expand_dims(res, axis=1)


array([[0.19052224],
       [0.17794971],
       [0.16731141],
       [0.16054159],
       [0.15667311],
       [0.14700193]])

In [ ]:
log = [1.38778666, 1.44185389, 1.49314718, 1.52270598, 1.54193734, 1.58845736]
log2 =  [3.69037176, 3.74443898, 3.79573227, 3.82529108, 3.84452244, 3.89104245]
norm.pdf(log2)

array([0.00044017, 0.00036002, 0.00029672, 0.00026511, 0.00024626,
       0.00020571])

1: 48.057
2. 64.226
m. 56.305

In [ ]:
stocks = norm.rvs(loc=0.09,scale=0.19,size=30, random_state=13)
bonds = norm.rvs(loc=0.04,scale=0.03, size=30, random_state=13)
returns = np.array([stocks,bonds]).T
weights = np.array([0.5,0.5])
porfolio_returns = returns @ weights
var = weights.T @ np.cov(returns.T) @ weights
cov = np.cov(stocks, porfolio_returns)[0,1]
beta = cov/var
porfolio_returns.mean()*beta





0.146